In [51]:
import pandas as pd
import numpy as np
import os
from glob import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# --- 1. إعداد البيانات (نفس الكود السابق مع التأكد من النوع) ---
base_path = r'C:\Users\haama\PycharmProjects\machine-learning-basics\Skin-Disease-Analyzer\data'
metadata = pd.read_csv(os.path.join(base_path, 'HAM10000_metadata.csv'))
all_image_paths = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(base_path, '**', '*.jpg'), recursive=True)}
metadata['path'] = metadata['image_id'].map(all_image_paths)
metadata.dropna(subset=['path'], inplace=True)
metadata = metadata.reset_index(drop=True)

metadata['age'] = metadata['age'].fillna(metadata['age'].mean())
meta_df = pd.get_dummies(metadata[['age', 'sex', 'localization']])
meta_array = meta_df.values.astype('float32')
le = LabelEncoder()
labels = le.fit_transform(metadata['dx']).astype('int32')
train_idx, test_idx = train_test_split(np.arange(len(metadata)), test_size=0.2, random_state=42)

# --- 2. بناء النموذج مع تسمية المدخلات لربطها بالمولد ---
num_meta = meta_array.shape[1]
img_input = layers.Input(shape=(128, 128, 3), name="img_input") # أعطيناه اسماً
base = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=(128, 128, 3))(img_input)
x_img = layers.GlobalAveragePooling2D()(base)

meta_input = layers.Input(shape=(num_meta,), name="meta_input") # أعطيناه اسماً
x_meta = layers.Dense(16, activation='relu')(meta_input)

merged = layers.Concatenate()([x_img, x_meta])
output = layers.Dense(7, activation='softmax')(layers.Dense(64, activation='relu')(merged))

hybrid_model = models.Model(inputs=[img_input, meta_input], outputs=output)
hybrid_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# --- 3. المولد المحدث (التعديل الجوهري هنا) ---
def final_safe_generator(indices, batch_size=32):
    while True:
        np.random.shuffle(indices)
        for i in range(0, len(indices), batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_imgs, batch_metas, batch_labels = [], [], []

            for idx in batch_indices:
                try:
                    img = img_to_array(load_img(metadata.iloc[idx]['path'], target_size=(128, 128))) / 255.0
                    batch_imgs.append(img)
                    batch_metas.append(meta_array[idx])
                    batch_labels.append(labels[idx])
                except: continue

            if len(batch_imgs) > 0:
                # نرسل البيانات كـ Dictionary لربطها بالأسماء التي وضعناها في النموذج
                yield ({"img_input": np.array(batch_imgs), "meta_input": np.array(batch_metas)}, np.array(batch_labels))

# --- 4. بدء التدريب ---
print("🚀 يبدأ التدريب الآن...")
train_gen = final_safe_generator(train_idx)
test_gen = final_safe_generator(test_idx)

hybrid_model.fit(
    train_gen,
    steps_per_epoch=len(train_idx) // 32,
    epochs=10,
    validation_data=test_gen,
    validation_steps=len(test_idx) // 32
)

🚀 يبدأ التدريب الآن...
Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 270s 993ms/step - accuracy: 0.6895 - loss: 0.8957 - val_accuracy: 0.6673 - val_loss: 3.4227
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 248s 994ms/step - accuracy: 0.7953 - loss: 0.5767 - val_accuracy: 0.0307 - val_loss: 5.3149
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 249s 995ms/step - accuracy: 0.8399 - loss: 0.4554 - val_accuracy: 0.1205 - val_loss: 4.8423
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 231s 926ms/step - accuracy: 0.8410 - loss: 0.4387 - val_accuracy: 0.6657 - val_loss: 3.2275
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 233s 933ms/step - accuracy: 0.8669 - loss: 0.3716 - val_accuracy: 0.6672 - val_loss: 4.2581
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 228s 912ms/step - accuracy: 0.8812 - loss: 0.3204 - val_accuracy: 0.6362 - val_loss: 2.4367
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 213s 854ms/step - accuracy: 0.9055 - loss: 0.2803 - val_accuracy: 0.5809 - val_loss: 3.5216
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 215s 861ms/st

In [56]:
import pandas as pd
import numpy as np
import os
from glob import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 1. إعداد المسارات (نفس مسارك)
base_path = r'C:\Users\haama\PycharmProjects\machine-learning-basics\Skin-Disease-Analyzer\data'
metadata = pd.read_csv(os.path.join(base_path, 'HAM10000_metadata.csv'))
all_image_paths = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(base_path, '**', '*.jpg'), recursive=True)}
metadata['path'] = metadata['image_id'].map(all_image_paths)
metadata.dropna(subset=['path'], inplace=True)
metadata = metadata.reset_index(drop=True)

# 2. تجهيز البيانات الرقمية (العمر، الجنس، إلخ)
metadata['age'] = metadata['age'].fillna(metadata['age'].mean())
meta_df = pd.get_dummies(metadata[['age', 'sex', 'localization']])
meta_array = meta_df.values.astype('float32')
le = LabelEncoder()
labels = le.fit_transform(metadata['dx']).astype('int32')

# 3. حساب "أهمية" كل مرض (Class Weights) لحل مشكلة عدم التوازن
weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = dict(enumerate(weights))

# 4. المولد "الذكي" (يرسل الصورة + البيانات + الوزن) ليتجنب الأخطاء
def final_generator(indices, batch_size=32):
    while True:
        np.random.shuffle(indices)
        for i in range(0, len(indices), batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_imgs, batch_metas, batch_labels, batch_sample_weights = [], [], [], []
            for idx in batch_indices:
                try:
                    img = img_to_array(load_img(metadata.iloc[idx]['path'], target_size=(128, 128))) / 255.0
                    label = labels[idx]
                    batch_imgs.append(img)
                    batch_metas.append(meta_array[idx])
                    batch_labels.append(label)
                    batch_sample_weights.append(class_weights_dict[label])
                except: continue
            if len(batch_imgs) > 0:
                yield ({"img_input": np.array(batch_imgs), "meta_input": np.array(batch_metas)},
                       np.array(batch_labels),
                       np.array(batch_sample_weights))

# 5. بناء النموذج الهجين (أصلحنا التوصيلات هنا)
num_meta = meta_array.shape[1]
img_input = layers.Input(shape=(128, 128, 3), name="img_input")
base_model = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
base_model.trainable = False # تجميد لمنع الـ Overfitting

x_img = base_model(img_input)
x_img = layers.GlobalAveragePooling2D()(x_img)
x_img = layers.Dropout(0.5)(x_img)

meta_input = layers.Input(shape=(num_meta,), name="meta_input")
x_meta = layers.Dense(32, activation='relu')(meta_input)

merged = layers.Concatenate()([x_img, x_meta])
x = layers.Dense(128, activation='relu')(merged)
x = layers.Dropout(0.3)(x)
output = layers.Dense(7, activation='softmax')(x)

hybrid_model = models.Model(inputs=[img_input, meta_input], outputs=output)
hybrid_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 6. تقسيم البيانات والبدء فوراً
train_idx, test_idx = train_test_split(np.arange(len(metadata)), test_size=0.2, random_state=42)
train_gen = final_generator(train_idx)
test_gen = final_generator(test_idx)

print("🚀 يبدأ التدريب الآن بأفضل إعدادات...")
hybrid_model.fit(train_gen, steps_per_epoch=len(train_idx)//32, epochs=10,
                 validation_data=test_gen, validation_steps=len(test_idx)//32)

hybrid_model.save('skin_disease_best_model.h5')
print("✅ مبروك! المشروع جاهز والنتائج ستكون ممتازة.")

🚀 يبدأ التدريب الآن بأفضل إعدادات...
Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 144s 557ms/step - accuracy: 0.3158 - loss: 2.3430 - val_accuracy: 0.5439 - val_loss: 1.4924
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 137s 550ms/step - accuracy: 0.4426 - loss: 1.4445 - val_accuracy: 0.6210 - val_loss: 1.2563
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 138s 554ms/step - accuracy: 0.5216 - loss: 1.3398 - val_accuracy: 0.5615 - val_loss: 1.2769
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 139s 558ms/step - accuracy: 0.5224 - loss: 1.2709 - val_accuracy: 0.5363 - val_loss: 1.2895
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 98s 391ms/step - accuracy: 0.5278 - loss: 1.1384 - val_accuracy: 0.5824 - val_loss: 1.1454
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 94s 376ms/step - accuracy: 0.5384 - loss: 1.1247 - val_accuracy: 0.6327 - val_loss: 1.0992
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 110s 440ms/step - accuracy: 0.5562 - loss: 1.1169 - val_accuracy: 0.5769 - val_loss: 1.1312
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1

✅ مبروك! المشروع جاهز والنتائج ستكون ممتازة.
